In [15]:
import requests 
import pandas as pd
import seaborn as sns
import numpy as np 
from datetime import datetime
from sqlalchemy import create_engine
from keys import *

In [16]:
global  bearer, full_directory_lst
full_directory_lst = []

url = f'https://id.twitch.tv/oauth2/token?client_id={my_client_id}&client_secret={my_client_secret}&grant_type=client_credentials'
x = requests.post(url)
bearer = x.text[17:47]


def get_directory():
    url = 'https://api.twitch.tv/helix/games/top'
    headers = { "Client-ID": f'{my_client_id}','Authorization': f'Bearer {bearer}',  'Accept': "application/vnd.twitchtv.v5+json", }
    resp = requests.get(url, headers = headers).json()

    for game in range(10):
        get_directory_viewers(resp['data'][game]['id'], resp['data'][game]['name'])


In [17]:
def get_directory_viewers(game_id, game_name):
    url= f'https://api.twitch.tv/helix/streams?first=100&game_id={game_id}'
    headers = { "Client-ID": f'{my_client_id}','Authorization': f'Bearer {bearer}',  'Accept': "application/vnd.twitchtv.v5+json", }
    resp = requests.get(url, headers = headers).json()
    pig=resp['pagination']['cursor']

    total = 0


    while len(resp['data']) > 1:
        for x in range(len(resp['data'])):
            total += resp['data'][x]['viewer_count']


        url= f'https://api.twitch.tv/helix/streams?after={pig}&first=100&game_id={game_id}'
        headers = { "Client-ID": f'{my_client_id}','Authorization': f'Bearer {bearer}',  'Accept': "application/vnd.twitchtv.v5+json", }
        resp = requests.get(url, headers = headers).json()
        try:
            pig=resp['pagination']['cursor']
        except:
            pig=''
    
    full_directory_lst.append([game_name, total])


In [18]:
def make_twitch_df(full_directory_lst):
    twitch_df = pd.DataFrame(columns=['Date', 'Time', 'Directory Name', 'Viewer Count'])
    
    date = str(datetime.now()).split(' ')[0]
    time = str(datetime.now()).split(' ')[1].split('.')[0]

    for item in full_directory_lst:
        twitch_df = twitch_df.append({'Date' : date, 'Time': time, 'Directory Name': item[0], 'Viewer Count' : item[1]},ignore_index=True )
    return twitch_df

In [19]:
def create_db():
    global engine
    # create sqlite engine
    engine = create_engine('sqlite:///twitch.db', echo=True)
    # create connection to engine
    conn = engine.connect()

    return conn

In [20]:
def add_to_db(twitch_df, conn):
    sqlite_table = "Twitch_Directory"
    twitch_df.to_sql(sqlite_table, conn, if_exists='append')


In [21]:
get_directory()
twitch_df = make_twitch_df(full_directory_lst)
conn = create_db()
add_to_db(twitch_df, conn)

2022-01-31 10:45:52,907 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2022-01-31 10:45:52,908 INFO sqlalchemy.engine.base.Engine ()
2022-01-31 10:45:52,909 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2022-01-31 10:45:52,909 INFO sqlalchemy.engine.base.Engine ()
2022-01-31 10:45:52,911 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Twitch_Directory")
2022-01-31 10:45:52,912 INFO sqlalchemy.engine.base.Engine ()
2022-01-31 10:45:52,913 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Twitch_Directory")
2022-01-31 10:45:52,913 INFO sqlalchemy.engine.base.Engine ()
2022-01-31 10:45:52,914 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Twitch_Directory" (
	"index" BIGINT, 
	"Date" TEXT, 
	"Time" TEXT, 
	"Directory Name" TEXT, 
	"Viewer Count" BIGINT
)


2022-01-31 10:45:52,915 INFO sqlalchemy.engine.base.Engine ()
2022-01-31 10:45:53,025 INFO sqlalchemy.engine.base.En

In [22]:
sql = """
SELECT *
FROM Twitch_Directory
"""
# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2022-01-31 10:45:53,271 INFO sqlalchemy.engine.base.OptionEngine 
SELECT *
FROM Twitch_Directory

2022-01-31 10:45:53,272 INFO sqlalchemy.engine.base.OptionEngine ()


,index,Date,Time,Directory Name,Viewer Count
0,0,2022-01-31,10:45:52,Just Chatting,390431
1,1,2022-01-31,10:45:52,Grand Theft Auto V,239160
2,2,2022-01-31,10:45:52,League of Legends,219827
3,3,2022-01-31,10:45:52,VALORANT,137060
4,4,2022-01-31,10:45:52,Fortnite,115131
